# Tech Intro

## Main tables

In [540]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [541]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = lead_list[17] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [542]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1120 00:20:46.184093 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1120 00:20:46.184939 10490 sched.cpp:241] Version: 1.11.0
I1120 00:20:46.185187 10528 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1120 00:20:46.185283 10528 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1120 00:20:46.186921 10530 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75602


## Read the spec-file from Hadoop and show some data sample

In [543]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+--------+--------------------+-----------+-------------+-------------------+---------------+----------------+---------+----------+---------------+---------------+---------------+---------------+----------------+
|id      |name                |date_create|last_modified|responsible_user_id|created_user_id|modified_user_id|filial_id|company_id|phone_generated|name1_generated|name2_generated|name3_generated|phone_normalized|
+--------+--------------------+-----------+-------------+-------------------+---------------+----------------+---------+----------+---------------+---------------+---------------+---------------+----------------+
|11076655|лиля                |null       |1665392570   |null               |null           |3790186         |1        |1         |               |лиля           |лиля           |лиля           |null            |
|11130665|Гуля                |null       |null         |null               |null           |null            |1        |1         |               |Г

## Show columns' properties

In [544]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- date_create: string (nullable = true)
 |-- last_modified: string (nullable = true)
 |-- responsible_user_id: string (nullable = true)
 |-- created_user_id: string (nullable = true)
 |-- modified_user_id: string (nullable = true)
 |-- filial_id: string (nullable = true)
 |-- company_id: string (nullable = true)
 |-- phone_generated: string (nullable = true)
 |-- name1_generated: string (nullable = true)
 |-- name2_generated: string (nullable = true)
 |-- name3_generated: string (nullable = true)
 |-- phone_normalized: string (nullable = true)



## Total rows (count)

In [545]:
eda_df.count()

625063

## Simple func to execute and show the result of the SQL-instruction

In [546]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [547]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [548]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|625063  |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [549]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,name,date_create,last_modified,responsible_user_id,created_user_id,modified_user_id,filial_id,company_id,phone_generated,name1_generated,name2_generated,name3_generated,phone_normalized
0,0,438,146023,150415,622620,292412,289844,806,0,438,438,438,438,520026


In [550]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,name,date_create,last_modified,responsible_user_id,created_user_id,modified_user_id,filial_id,company_id,phone_generated,name1_generated,name2_generated,name3_generated,phone_normalized
0,0,183,0,0,0,0,0,0,0,117,96,147,151,0


In [551]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,name,date_create,last_modified,responsible_user_id,created_user_id,modified_user_id,filial_id,company_id,phone_generated,name1_generated,name2_generated,name3_generated,phone_normalized
0,0,8484,0,0,0,0,0,0,0,494595,12700,97284,112016,0


In [552]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
name,438,183,8484
date_create,146023,0,0
last_modified,150415,0,0
responsible_user_id,622620,0,0
created_user_id,292412,0,0
modified_user_id,289844,0,0
filial_id,806,0,0
company_id,0,0,0
phone_generated,438,117,494595


In [553]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [554]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
name,0.070,0.029,1.357
date_create,23.361,0.000,0.000
last_modified,24.064,0.000,0.000
responsible_user_id,99.609,0.000,0.000
created_user_id,46.781,0.000,0.000
modified_user_id,46.370,0.000,0.000
filial_id,0.129,0.000,0.000
company_id,0.000,0.000,0.000
phone_generated,0.070,0.019,79.127


### SQL-instructions

In [555]:
#Find dupes or misunderstanding or useless static values 625 063 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|625063            |
+------------------+

+--------------------+
|count(DISTINCT name)|
+--------------------+
|197246              |
+--------------------+

+---------------------------+
|count(DISTINCT date_create)|
+---------------------------+
|468933                     |
+---------------------------+

+-----------------------------+
|count(DISTINCT last_modified)|
+-----------------------------+
|454238                       |
+-----------------------------+

+-----------------------------------+
|count(DISTINCT responsible_user_id)|
+-----------------------------------+
|17                                 |
+-----------------------------------+

+-------------------------------+
|count(DISTINCT created_user_id)|
+-------------------------------+
|11586                          |
+-------------------------------+

+--------------------------------+
|count(DISTINCT modified_user_id)|
+--------------------------------+

In [556]:
def execute_template(column: str ='id') -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "ID incl."
    :param :string select_instruction: column name
    :return: result of the SQL-operation as a table
    """
    print(execute("SELECT DISTINCT(" + column + ")\
                FROM eda_sql\
                WHERE 1 = 1 \
                ORDER BY " + column + " DESC"))
    return execute("SELECT DISTINCT(" + column + ")\
                    FROM eda_sql\
                    WHERE 1 = 1 \
                    ORDER BY " + column + " ASC")

In [557]:
#Find dupes or misunderstanding or useless static values 625 063 total
for col in eda_df.columns:
    execute_template(col)

+-------+
|id     |
+-------+
|9982181|
|9943669|
|9904539|
|9903901|
|9903667|
|9896645|
|9756943|
|9731949|
|9572987|
|9569741|
|951085 |
|947175 |
|9358031|
|9356789|
|9219869|
|9188321|
|9187103|
|9177179|
|9168943|
|9102633|
+-------+
only showing top 20 rows

None
+--------+
|id      |
+--------+
|10053613|
|10062929|
|10071263|
|10074673|
|10102823|
|10113025|
|10298937|
|10383005|
|10629197|
|10705867|
|1072713 |
|1079425 |
|10823795|
|10848913|
|10906519|
|10907411|
|10913867|
|10961343|
|10983383|
|1106223 |
+--------+
only showing top 20 rows

+------------------------------+
|name                          |
+------------------------------+
|﻿Фарзана/Динар                |
|﻿Насима Гайфуловна            |
|﻿Зульфира +79378511487        |
|﻿Альбина +79649576825         |
|№ 2                           |
|№                             |
|…                             |
|•\tОвчинников Сергей Федорович|
|яяяяяяяяя                     |
|яяяяяяя                       |
|яяяяя    

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [539]:
spark.stop()

I1120 00:20:39.770716 10390 sched.cpp:2174] Asked to stop the driver
I1120 00:20:39.770854 10587 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75601
